## Schema
Student(**sID**, sur, first, campus, email, cgpa)

Course (**dept, cNum**, name, br)

Offering(**oID**, dept, cNum, term, instructor)

Took (**sID, oID**, grade)

Offering[dept, cNum] $\subseteq$ Course[dept, cNum]

Took[sID]$\subseteq$ Student[sID]

Took[oID]$\subseteq$ Offering[oID]

#### Groupby-having.pdf

1. Find the average, min, max grade for each offerings.
```SQL
SELECT ave(grade), min(grade), max(grade)
FROM Took
GROUP BY oID
```

2. 
<table>
    <tr>
        <td>Invalid</td><td>Valid</td>
    </tr>
    <tr>
        <td>Invalid</td><td>Valid</td>
    </tr>
</table>
        
    
   

3. find the sid and min grade of each student with an average over 80

```SQL
SELECT sid, min(grade)
FROM Took
GROUP BY sID
HAVING avg(grade) > 80
```

4. find the sID, sur, and ave grade of each student, but keep the data only for those students who have taken at least 10 courses.

```SQL
SELECT sID, sur, ave(grade)
FROM Student, Took
WHERE Student.sID = Took.sID
GROUP BY Student.sID
HAVING count(Took) >= 10
```

5. For each student who has passed at least 10 courses, report their sid and average grade on the course that they passed. 

```SQL
SELECT sid, ave(grade)
FROM Took
WHERE grade >= 50 
GROUP BY sid
HAVING count(*) >= 10 
```

6. For each student who has passed at least 10 courses, report their sID and average grade on all of their courses.

```SQL
CREATE VIEW Seniors AS
SELECT sID
FROM Took
WHERE grade >= 50
GROUP BY sID
HAVING count(*) >= 10

SELECT ave(grade)
FROM Took, Seniors
WHERE Took.sID = Seniors.sID
GROUP BY Seniors.sID
```

7. <table>
    <tr>
        <td>Valid</td><td>Valid</td>
    </tr>
    <tr>
        <td>Invalid</td><td>Invalid</td>
    </tr>
</table>
        
    

#### Joins.pdf

1. 
(a) Valid <br>
(b) Invalid (can't have two distinct together in select)<br>
(c) Valid

3. For each student who has taken a course, report their sid and the number of different department they have taken a course in. 

```SQL
SELECT sID, count(distinct dept)
FROM Took t, Offering o
WHERE t.oID = o.oID
GROUP BY sID
```

4. One
<table>
        <tr>
                <th>a</th><th>b</th>
        </tr>
        <tr>
                <td>1</td><td>2</td>
        </tr>
        <tr>
                <td>6</td><td>12</td>
        </tr>
        <tr>
                <td>/</td><td>100</td>
        </tr>
        <tr>
                <td>20</td><td>/</td>
        </tr>
</table>
Two
<table>
        <tr>
                <th>b</th><th>c</th>
        </tr>
        <tr>
                <td>2</td><td>3</td>
        </tr>
        <tr>
                <td>100</td><td>101</td>
        </tr>
        <tr>
                <td>20</td><td>21</td>
        </tr>
        <tr>
                <td>2</td><td>4</td>
        </tr>
        <tr>
                <td>2</td><td>5</td>
        </tr>
</table>

a) For what query could produce this result?
<table>
        <tr>
                <th>a</th><th>b</th><th>c</th><th></th>
        </tr>
        <tr>
                <td>1</td><td>2</td><td>3</td>
        </tr>
        <tr>
                <td>1</td><td>2</td><td>4</td>
        </tr>
        <tr>
                <td>1</td><td>2</td><td>5</td>
        </tr>
        <tr>
                <td>/</td><td>20</td><td>21</td>
        </tr>
        <tr>
                <td>/</td><td>100</td><td>101</td>
        </tr>
</table>

```SQL
SELECT *
FROM One NATURAL RIGHT JOIN Two
```

b) <table>
        <tr>
                <th>a</th><th>b</th><th>c</th>
        </tr>
        <tr>
                <td>1</td><td>2</td><td>3</td>
        </tr>
        <tr>
                <td>1</td><td>2</td><td>4</td>
        </tr>
        <tr>
                <td>1</td><td>2</td><td>5</td>
        </tr>
        <tr>
                <td>6</td><td>12</td><td>/</td>
        </tr>
        <tr>
                <td>/</td><td>100</td><td>101</td>
        </tr>
        <tr>
                <td>20</td><td>/</td><td>/</td>
        </tr>
</table>

```SQL
SELECT *
FROM One NATURAL LEFT JOIN Two
```

#### Subqueries.pdf

1. 

```SQL
# "||" concatenate strings
SELECT sid, dept || cum as course, grade 
FROM Took, 
    (SELECT *
     FROM Offering
     WHERE instructor = 'Horton') Hoffering
WHERE Took.oid = Hoffering.oid
```

This query: report the id and course of students who took courses taught by Horton

2. 
```SQL
SELECT sid, surname
FROM Student
WHERE cgpa > 
    (SELECT cgpa
    FROM Student
    WHERE sid = 99999);
```
Find all students having cgpa higher than student with ID 99999, and report their ID and surname

3. ```SQL
SELECT sid, dept||cum AS course, grade
FROM Took JOIN Offering ON Took.oid = Offering.oid
WHERE grade >= 80 AND
    (cNum, dept) IN (
        SELECT cNum, dept
        FROM Took JOIN Offering ON Took.oid=OFfering.oid 
        JOIN Student ON Took.sid = Student.oid
        WHERE surname = "Lakemeyer")
```
find students who took a course Lakemeyer ever took and had mark over 80, report their sid and courses.


4. ```SQL
SELECT a
FROM R
WHERE b in (SELECT b FROM s)
```
is equivalent to 

```SQL
SELECT a 
FROM R, S
WHERE R.b = S.b
```

5. ```SQL
SELECT instructor
FROM OFfering O1
WHERE NOT EXISTS (
    SELECT *
    FROM Offering
    WHERE  oid <> O1.oid 
    AND instructor = O1.instructor);
```

Select all instructor that only teach exactly one course

6. ```SQL
SELECT DISTINCT oid
FROM Took
WHERE EXISTS (
    SELECT *
    FROM Took t, Offering o
    WHERE
        t.oid = o.oid AND
        t.pod <> Took.oid AND
        o.dept = 'CSC' AND
        Took.sid = t.sid )
ORDER BY oid;
```

It finds course offerings that include a student who has taken something else that is a CSC course.


7. 
```SQL
CREATE VIEW Counts AS
SELECT dept, cNum, instructor, count(oID)
FROM Offering
GROUP BY dept, cNum, instructor
```